In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from pandarallel import pandarallel
from langdetect import detect, DetectorFactory


In [ ]:
#Reading the cleaning data set, insert own path to file 995,000_rows.csv
df = pd.read_csv("995,000_rows.csv")

C:\Users\Danie\AppData\Local\Temp\ipykernel_11404\2103519170.py:2: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("C:/Users/Danie/OneDrive/Skrivebord/FN_Directory/CSV/995,000_rows.csv")


In [18]:
from pandarallel import pandarallel
#DetectorFactory.seed = 0

# Initialiser pandarallel
pandarallel.initialize(progress_bar=True)  #giver os en progress
  # kan også manuelt indstilles

def detect_language():
  from langdetect import detect, DetectorFactory
  DetectorFactory.seed = 0
  df['content']=df['content'].astype(str).fillna("")
  df['language']=df['content'].parallel_apply(lambda x: detect(x))

detect_language()

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [19]:
df['language'].value_counts()


language
en       977844
ru        11797
fr         1381
es         1348
el          879
it          542
de          476
ar          359
nl           84
pt           70
vi           65
no           33
da           24
tl           19
ca           15
et           14
af           11
cy           10
sv            7
id            3
zh-cn         3
ro            2
ko            2
sk            2
he            2
pl            2
bg            1
hu            1
hi            1
so            1
hr            1
lt            1
Name: count, dtype: int64

In [20]:
df_en = df[df['language'] == 'en']

In [21]:
df_en['language'].value_counts()

language
en    977844
Name: count, dtype: int64

In [22]:
df['type'].value_counts()

type
reliable                      218564
political                     194518
bias                          133232
fake                          104883
conspiracy                     97314
rumor                          56445
unknown                        43534
unreliable                     35332
clickbait                      27412
junksci                        14040
satire                         13160
hate                            8779
2018-02-10 13:43:39.521661         1
Name: count, dtype: int64

In [25]:
#Calcuate the percentage of non-english articles in the total corpus
count_non_english = len(df[df['language'] != 'en'])
count_english = len(df[df['language'] == 'en'])
percent = (count_non_english / count_english)*100
print("Percentage of non-English articles in the corpus:", percent)


Percentage of non-English articles in the corpus: 1.7544720834816188


### Giving every type a broad type, so all articles are marked as either Fake or Reliable news.

In [26]:
#Assigning every type a new category, so there only are two types 'Fake News' or 'Reliable News'
category_mapping = {
    "fake": "Fake News",
    "satire": "Fake News",
    "bias": "Fake News",
    "conspiracy": "Fake News",
    "junksci": "Fake News",
    "hate": "Reliable News",
    "clickbait": "Reliable News",
    "unreliable": "Fake News",
    "political": "Reliable News",
    "reliable": "Reliable News",  
    "rumor": "Fake News"
}

#Appling the mapping to the 'type' column, making a new column called 'broad_category'
df['broad_category'] = df['type'].map(category_mapping)

In [27]:
#Removing rows with NaN-values in 'broad_category'
df = df.dropna(subset=['broad_category'])

In [28]:
#Article with no broad category is removed
df = df[~df["broad_category"].isin(['unknown']) & df["broad_category"].notna()]

In [29]:
df['broad_category'].value_counts()

broad_category
Fake News        454406
Reliable News    449273
Name: count, dtype: int64

In [ ]:
#Splitting data into training set (80%) and the rest (20%) 
train, temp = train_test_split(df, test_size=0.2, random_state=0)

#Splitting the rest into validation set(50% of 20% = 10%) and testset (50% of 20% = 10%)
val, test = train_test_split(temp, test_size=0.5, random_state=0)

#Size of the three sets
print(f"Training data: {len(train)} rows")
print(f"Validation: {len(val)} rows")
print(f"Test: {len(test)} rows")


#Converting the sets ad csv-files
train.to_csv('full_train_set.csv', index=False)
val.to_csv('full_val_set.csv', index=False)
test.to_csv('full_test_set.csv', index=False)


Training data: 722943 rows
Validation: 90368 rows
Test: 90368 rows
